In [4]:
import glob
import pathlib
import shutil
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import pathlib
from statistics import mean

import torch
from scipy import linalg
#from scipy.misc import imread
from matplotlib.pyplot import imread
from torch.nn.functional import adaptive_avg_pool2d

from tqdm import tqdm
from inception import InceptionV3
import torchvision
import numpy
import scipy
import pickle

In [5]:
path1 = pathlib.Path('./Image-coco/cover/')
files1 = list(sorted(path1.glob('*')))
print(files1)

[WindowsPath('Image-coco/cover/dc00000.png'), WindowsPath('Image-coco/cover/dc00001.png'), WindowsPath('Image-coco/cover/dc00002.png'), WindowsPath('Image-coco/cover/dc00004.png'), WindowsPath('Image-coco/cover/dc00006.png'), WindowsPath('Image-coco/cover/dc00007.png'), WindowsPath('Image-coco/cover/dc00008.png'), WindowsPath('Image-coco/cover/dc00009.png'), WindowsPath('Image-coco/cover/dc00010.png'), WindowsPath('Image-coco/cover/dc00012.png'), WindowsPath('Image-coco/cover/dc00013.png'), WindowsPath('Image-coco/cover/dc00014.png'), WindowsPath('Image-coco/cover/dc00015.png'), WindowsPath('Image-coco/cover/dc00016.png'), WindowsPath('Image-coco/cover/dc00018.png'), WindowsPath('Image-coco/cover/dc00019.png'), WindowsPath('Image-coco/cover/dc00021.png'), WindowsPath('Image-coco/cover/dc00022.png'), WindowsPath('Image-coco/cover/dc00023.png'), WindowsPath('Image-coco/cover/dc00024.png'), WindowsPath('Image-coco/cover/dc00025.png'), WindowsPath('Image-coco/cover/dc00026.png'), WindowsPa

In [6]:
trg_path = pathlib.Path("./realsx25/")

name_list = []
for k in range(25):
    name_list.append('_gen_sample_' + str(k) + '.png')

i=0

for n in range(len(files1)):
    for m in sorted(name_list):
        shutil.copy(str(files1[n]), trg_path.joinpath('%.5d' % i + m))
        i += 1    

In [7]:
def get_activations(files, model, batch_size=1, dims=64,
                    cuda=False, verbose=False):
    """Calculates the activations of the pool_3 layer for all images.

    Params:
    -- files       : List of image files paths
    -- model       : Instance of inception model
    -- batch_size  : Batch size of images for the model to process at once.
                     Make sure that the number of samples is a multiple of
                     the batch size, otherwise some samples are ignored. This
                     behavior is retained to match the original FID score
                     implementation.
    -- dims        : Dimensionality of features returned by Inception
    -- cuda        : If set to True, use GPU
    -- verbose     : If set to True and parameter out_step is given, the number
                     of calculated batches is reported.
    Returns:
    -- A numpy array of dimension (num images, dims) that contains the
       activations of the given tensor when feeding inception with the
       query tensor.
    """
    model.eval()

    if len(files) % batch_size != 0:
        print(('Warning: number of images is not a multiple of the '
               'batch size. Some samples are going to be ignored.'))
    if batch_size > len(files):
        print(('Warning: batch size is bigger than the data size. '
               'Setting batch size to data size'))
        batch_size = len(files)

    n_batches = len(files) // batch_size
    n_used_imgs = n_batches * batch_size

    pred_arr = np.empty((n_used_imgs, dims))

    for i in tqdm(range(n_batches)):
        if verbose:
            print('\rPropagating batch %d/%d' % (i + 1, n_batches),
                  end='', flush=True)
        start = i * batch_size
        end = start + batch_size

        images = np.array([imread(str(f)).astype(np.float32)
                           for f in files[start:end]])

        images = images[:, :, :, 0:3]
        # Reshape to (n_images, 3, height, width)
        images = images.transpose((0, 3, 1, 2))
        # images = images[0,:,:,:]
        # images /= 255

        batch = torch.from_numpy(images).type(torch.FloatTensor)
        if cuda:
            batch = batch.cuda()

        pred = model(batch)[0]

        # If model output is not scalar, apply global spatial average pooling.
        # This happens if you choose a dimensionality not equal 2048.

        # if pred.shape[2] != 1 or pred.shape[3] != 1:
        #    pred = adaptive_avg_pool2d(pred, output_size=(1, 1))

        pred_arr = pred.cpu().data.numpy().transpose(0, 2, 3, 1).reshape(batch_size * pred.shape[2] * pred.shape[3], -1)

    if verbose:
        print(' done')

    return pred_arr


def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """Numpy implementation of the Frechet Distance.
    The Frechet distance between two multivariate Gaussians X_1 ~ N(mu_1, C_1)
    and X_2 ~ N(mu_2, C_2) is
            d^2 = ||mu_1 - mu_2||^2 + Tr(C_1 + C_2 - 2*sqrt(C_1*C_2)).

    Stable version by Dougal J. Sutherland.

    Params:
    -- mu1   : Numpy array containing the activations of a layer of the
               inception net (like returned by the function 'get_predictions')
               for generated samples.
    -- mu2   : The sample mean over activations, precalculated on an
               representative data set.
    -- sigma1: The covariance matrix over activations for generated samples.
    -- sigma2: The covariance matrix over activations, precalculated on an
               representative data set.

    Returns:
    --   : The Frechet Distance.
    """

    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, \
        'Training and test mean vectors have different lengths'
    assert sigma1.shape == sigma2.shape, \
        'Training and test covariances have different dimensions'

    diff = mu1 - mu2

    # Product might be almost singular
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        msg = ('fid calculation produces singular product; '
               'adding %s to diagonal of cov estimates') % eps
        print(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    # Numerical error might give slight imaginary component
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError('Imaginary component {}'.format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return (diff.dot(diff) + np.trace(sigma1) +
            np.trace(sigma2) - 2 * tr_covmean)


def calculate_activation_statistics(files, model, batch_size=1,
                                    dims=64, cuda=False, verbose=False):
    """Calculation of the statistics used by the FID.
    Params:
    -- files       : List of image files paths
    -- model       : Instance of inception model
    -- batch_size  : The images numpy array is split into batches with
                     batch size batch_size. A reasonable batch size
                     depends on the hardware.
    -- dims        : Dimensionality of features returned by Inception
    -- cuda        : If set to True, use GPU
    -- verbose     : If set to True and parameter out_step is given, the
                     number of calculated batches is reported.
    Returns:
    -- mu    : The mean over samples of the activations of the inception model.
    -- sigma : The covariance matrix of the activations of the inception model.
    """
    act = get_activations(files, model, batch_size, dims, cuda, verbose)
    mu = np.mean(act, axis=0)
    sigma = np.cov(act, rowvar=False)
    return mu, sigma


def _compute_statistics_of_path(files, model, batch_size, dims, cuda):
    if path.endswith('.npz'):
        f = np.load(path)
        m, s = f['mu'][:], f['sigma'][:]
        f.close()
    else:
        path = pathlib.Path(path)
        files = list(path.glob('*.jpg'))+ list(path.glob('*.png'))
#         files = list(path.glob('*/*'))
        m, s = calculate_activation_statistics(files, model, batch_size,
                                               dims, cuda)

    return m, s


def calculate_sifid_given_paths(path1, path2, batch_size, cuda, dims, suffix):
    """Calculates the SIFID of two paths"""

    block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[dims]

    model = InceptionV3([block_idx])
    if cuda:
        model.cuda()

    path1 = pathlib.Path(path1)
    files1 = list(path1.glob('*'))

    path2 = pathlib.Path(path2)
    files2 = list(path2.glob('*'))

    fid_values = []
    Im_ind = []
    for i in range(len(files2)):
        m1, s1 = calculate_activation_statistics([files1[i]], model, batch_size, dims, cuda)
        m2, s2 = calculate_activation_statistics([files2[i]], model, batch_size, dims, cuda)
        fid_values.append(calculate_frechet_distance(m1, s1, m2, s2))
        file_num1 = files1[i].name
        file_num2 = files2[i].name
#         Im_ind.append(int(file_num1[:-4]))
#         Im_ind.append(int(file_num2[:-4]))
        Im_ind.append(int(file_num1[:5]))
        Im_ind.append(int(file_num2[:5]))

    print(Im_ind)
    return fid_values

In [8]:
path1 = './realsx25'
path2 = './gen_samples'
suffix = 'png'
sifid_values = calculate_sifid_given_paths(path1,path2,1,''!='',64,suffix)

sifid_values = np.asarray(sifid_values,dtype=np.float32)
numpy.save('SIFID', sifid_values)
print('SIFID: ', sifid_values.mean())

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 125.34it/s]


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 91.14it/s]


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 125.34it/s]


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 125.34it/s]


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 287.93it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 91.15it/s]


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 111.41it/s]


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 111.41it/s]


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 111.41it/s]


[0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36, 37, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 42, 43, 43, 44, 44, 45, 45, 46, 46, 47, 47, 48, 48, 49, 49, 50, 50, 51, 51, 52, 52, 53, 53, 54, 54, 55, 55, 56, 56, 57, 57, 58, 58, 59, 59, 60, 60, 61, 61, 62, 62, 63, 63, 64, 64, 65, 65, 66, 66, 67, 67, 68, 68, 69, 69, 70, 70, 71, 71, 72, 72, 73, 73, 74, 74, 75, 75, 76, 76, 77, 77, 78, 78, 79, 79, 80, 80, 81, 81, 82, 82, 83, 83, 84, 84, 85, 85, 86, 86, 87, 87, 88, 88, 89, 89, 90, 90, 91, 91, 92, 92, 93, 93, 94, 94, 95, 95, 96, 96, 97, 97, 98, 98, 99, 99, 100, 100, 101, 101, 102, 102, 103, 103, 104, 104, 105, 105, 106, 106, 107, 107, 108, 108, 109, 109, 110, 110, 111, 111, 112, 112, 113, 113, 114, 114, 115, 115, 116, 116, 117, 117, 118, 118, 119, 119, 120, 120, 121, 121,